# Tomopy Reconstruction Demo for ALS at ALCF

This notebook demonstrates how to perform tomography reconstructions using Tomopy for ALS data on Polaris at ALCF using [Globus Flows](https://www.globus.org/globus-flows-service).  In this example, Globus flows will launch the application on Polaris and then transfer results from the Eagle filesystem.

This notebook can be run from anywhere, it only requires a local installation of Globus software (described below) and access to a Globus Compute Endpoint setup by the user on Polaris that has access to tomopy (also described below).

This demo uses Globus Flows and Globus Compute.  Globus Flows is a reliable and secure platform for orchestrating and performing research data management and analysis tasks. A flow is often needed to manage data coming from instruments, e.g., image files can be moved from local storage attached to a microscope to a high-performance storage system where they may be accessed by all members of the research project.  Globus Compute is a remote executor for tasks expressed as python functions that are sent to remote machines following a fire-and-forget model.

In this notebook we will first describe necessary setup tasks for the local environment and on Polaris; second, we will describe how to create and test a Globus Compute function that can remotely launch a tomopy task on Polaris compute nodes; and third, we will describe how to incorporate this function with a Globus Flow that coordinates the execution of the tomopy task with a data transfer step.

More examples of creating and running Globus Flows can be found on Globus' [demo instance](https://jupyter.demo.globus.org/hub/).


## Tomopy on Polaris

Tomopy has been installed in a conda environment on Polaris at this path which is accessible to members of the IRIBeta allocation: `/eagle/IRIBeta/als/env/tomopy`.

## Local Setup

This notebook can be run from anywhere.  The only requirement is a local environment, such as a conda environment, that has python 3.11 installed along with the Globus packages `globus_compute_sdk` and `globus_cli`.  If you have a local installation of conda you can set up an environment that can run this notebook with these steps:

```bash
conda create -n globus_env python==3.11
conda activate globus_env
pip install globus_compute_sdk globus_cli
```

Note that the tomopy environment on Polaris contains python 3.11. It is therefore necessary for this environment on your local machine to have a python version close to this version.

## Create a Globus Compute Endpoint on Polaris

The first step for a user to execute applications on Polaris through the Globus service is to create a Globus compute endpoint on Polaris.  This requires the user to do a one-time setup task to configure the endpoint.

In a shell seperate from this notebook, log into Polaris.  Copy the file included with this notebook called `template_config.yaml` to the Polaris filesystem (doesn't matter where).  Inside `template_config.yaml` you should see options setting your project name (`IRIBeta`), the queue you will use (`debug`), and commands that activate a pre-made conda environment on Polaris that can run tomopy.

In your shell on Polaris, execute the following commands:

```bash
module load conda
conda activate /eagle/IRIBeta/als/env/tomopy
globus-compute-endpoint configure --endpoint-config template_config.yaml als_endpoint
globus-compute-endpoint start als_endpoint
globus-compute-endpoint list
```
This will create an endpoint and display its status.  Its status should be listed as `running`.  There will also be displayed a unique Endpoint ID in the form of a UUID.  Copy that ID and paste it below as a string assigned to `YOUR_ENDPOINT_FROM_ABOVE`.

In [ ]:
YOUR_ENDPOINT_FROM_ABOVE = "UUID-HERE"
polaris_endpoint_id = YOUR_ENDPOINT_FROM_ABOVE

Your endpoint is now active as a daemon process running on the Polaris login node.  It is communicating with the Globus service and waiting for work.  If you ever want to stop the process you can run:
```bash
globus-compute-endpoint stop als_endpoint
```
Your process may need to be periodically restarted, for example after Polaris comes back from a maintance period.

If you ever need to make changes to your endpoint configuration, you can find the settings file in `~/.globus_compute/als_endpoint/config.yaml`.  Edit this file and then restart the endpoint with `globus-compute-endpoint restart als_endpoint` to make the changes active.

This endpoint will submit work to the `debug` queue since this demo is for learning purposes.  In production, ALS will be able to submit work to the [demand queue](https://docs.alcf.anl.gov/polaris/running-jobs/#queues) which will give immediate access to Polaris compute nodes.

## Create a Function

We first need to create a python function that wraps around the application call.  We will call it `reconstruction_wrapper` that takes as an input the directory on the eagle file system where the input data are located, `rundir`, and the name of the input `parametersfile`.

In [ ]:
# Original code
def reconstruction_wrapper(rundir, parametersfile="inputOneSliceOfEach.txt"):
    import os
    import time
    import subprocess

    start = time.time()

    # Move to directory where data are located
    os.chdir(rundir)

    # Run reconstruction.py
    command = f"python /eagle/IRIBeta/als/example/reconstruction.py {parametersfile}"
    res = subprocess.run(command.split(" "), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    end = time.time()
    
    return f"Reconstructed data specified in {parametersfile} in {end-start} seconds;\n {res}"

In [ ]:
# Updated script for globus_reconstruction.py
def reconstruction_wrapper(rundir, file_name, folder_path):
    import os
    import time
    import subprocess

    start = time.time()

    # Move to directory where data are located
    os.chdir(rundir)

    # Run reconstruction.py
    command = f"python /eagle/IRIBeta/als/example/globus_reconstruction.py {file_name} {folder_path}"
    res = subprocess.run(command.split(" "), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    end = time.time()
    
    return f"Reconstructed data specified in {folder_path} / {file_name} in {end-start} seconds;\n {res}"

In [ ]:
# Updated flow for globus_reconstruction.py + zarr generation
def reconstruction_wrapper(rundir, h5_file_name, folder_path):
    import os
    import time
    import subprocess

    rec_start = time.time()

    # Move to directory where data are located
    os.chdir(rundir)

    # Run reconstruction.py
    command = f"python /eagle/IRIBeta/als/example/globus_reconstruction.py {h5_file_name} {folder_path}"
    recon_res = subprocess.run(command.split(" "), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    rec_end = time.time()
    
    print(f"Reconstructed data specified in {folder_path} / {h5_file_name} in {rec_end-rec_start} seconds;\n {recon_res}")

    start = time.time()

    # Convert tiff files to zarr
    file_name = h5_file_name[:-3] if h5_file_name.endswith('.h5') else h5_file_name
    command = f"python /eagle/IRIBeta/als/example/tiff_to_zarr.py /eagle/IRIBeta/als/bl832/scratch/{folder_path}/rec{file_name}/" # --zarr_directory /path/to/storage/"
    zarr_res = subprocess.run(command.split(" "), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    end = time.time()

    print(f"Converted tiff files to zarr in {end-start} seconds;\n {zarr_res}")

    return f"Reconstructed data specified in {folder_path} / {h5_file_name} in {rec_end-rec_start} seconds;\n {recon_res} \n Converted tiff files to zarr in {end-start} seconds;\n {zarr_res}"

This notebook can be run from anywhere.  The only requirement is a local environment, such as a conda environment, that has python 3.11 installed along with the Globus packages `globus_compute_sdk` and `globus_cli`.  If you have a local installation of conda you can set up an environment that can run this notebook with these steps:

```bash
conda create -n globus_env python==3.11
conda activate globus_env
pip install globus_compute_sdk globus_cli python-dotenv
```

Note that the tomopy environment on Polaris contains python 3.11. It is therefore necessary for this environment on your local machine to have a python version close to this version.

## Authenticate Client and Test Function

We will now instantiate a Globus Compute client to test the function.  Globus will prompt the user for their credentials if running for the first time.  The user should have a Globus account through their ALCF account and should validate with their ALCF credentials.

In [ ]:
from globus_compute_sdk import Client, Executor
import os
os.environ["GLOBUS_COMPUTE_CLIENT_ID"] = os.getenv("GLOBUS_CLIENT_ID")
os.environ["GLOBUS_COMPUTE_CLIENT_SECRET"] = os.getenv("GLOBUS_CLIENT_SECRET")

gc = Client()

polaris_endpoint_id = YOUR_ENDPOINT_FROM_ABOVE
gce = Executor(endpoint_id=polaris_endpoint_id)

In [ ]:
# Original code for submitting the job
future = gce.submit(reconstruction_wrapper, "/eagle/IRIBeta/als/example")
print(future.result())

In [ ]:
# Updated code for globus_reconstruction.py
future = gce.submit(reconstruction_wrapper, "/eagle/IRIBeta/als/bl832/raw", "20230224_132553_sea_shell.h5", "BLS-00564_dyparkinson")
print(future.result())

This notebook can be run from anywhere.  The only requirement is a local environment, such as a conda environment, that has python 3.11 installed along with the Globus packages `globus_compute_sdk` and `globus_cli`.  If you have a local installation of conda you can set up an environment that can run this notebook with these steps:

```bash
conda create -n globus_env python==3.11
conda activate globus_env
pip install globus_compute_sdk globus_cli python-dotenv
```

Note that the tomopy environment on Polaris contains python 3.11. It is therefore necessary for this environment on your local machine to have a python version close to this version.

## Register Function

Now that the function has been tested and works, register the function with the Globus service.  This will allow the user to call the function from within a flow.

In [ ]:
reconstruction_func = gc.register_function(reconstruction_wrapper)

print(reconstruction_func)

In [ ]:
# Original code
future = gce.submit_to_registered_function(args=["/eagle/IRIBeta/als/example"], function_id=reconstruction_func)
future.result()

In [ ]:
# Updated code for globus_reconstruction.py
future = gce.submit_to_registered_function(args=["/eagle/IRIBeta/als/bl832/raw", "20230224_132553_sea_shell.h5", "BLS-00564_dyparkinson"], function_id=reconstruction_func)
future.result()

## Incorporate Function into a Flow

Now we will incorporate the Tomopy function into an example flow to run Tomopy on Polaris in coordination with other tasks.

This example simply includes two steps:
1. Run Tomopy via Globus Compute
2. Transfer results from the eagle file system to the home file system.

This can easily be extended to include other steps to import data, perform postprocessing, or publish and catalog results.

This is the flow definition for this two-step flow.

In [ ]:
flow_definition = {
    "Comment": "Run Reconstruction and transfer results",
    "StartAt": "Reconstruction",
    "States": {
        "Reconstruction": {
            "Comment": "Reconstruction with Tomopy",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/fxap",
            "Parameters": {
                "endpoint.$": "$.input.compute_endpoint_id",
                "function.$": "$.input.compute_function_id",
                "kwargs.$": "$.input.compute_function_kwargs"
            },
            "ResultPath": "$.ReconstructionOutput",
            "WaitTime": 3600,
            "Next": "Transfer_Out"
        },
        "Transfer_Out": {
            "Comment": "Transfer files",
            "Type": "Action",
            "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
            "Parameters": {
                "source_endpoint_id.$": "$.input.source.id",
                "destination_endpoint_id.$": "$.input.destination.id",
                "transfer_items": [
                    {
                        "source_path.$": "$.input.source.path",
                        "destination_path.$": "$.input.destination.path",
                        "recursive.$": "$.input.recursive_tx"
                    }
                ]
            },
            "ResultPath": "$.TransferFiles",
            "WaitTime": 300,
            "End": True
        },
    }
}

Next, we need to provide a flows client id to run the flow.  For now we will use the demo client id, but a project should create a client for their work.

In [ ]:
import os
import time
import globus_sdk

# from utils import get_flows_client, get_specific_flow_client
from orchestration.globus.flows import get_flows_client, get_specific_flow_client

# Tutorial client ID
# We recommend replacing this with your own client for any production use-cases
# Create your own at developers.globus.org
CLIENT_ID = "UUID-HERE"

Now get an instance of the flows client.  You will be asked to validate credentials with the Globus service.  

In [ ]:
fc = get_flows_client()

Next, create a flow.  You will again be asked to validate credentials with the globus service.

In [ ]:
flow = fc.create_flow(definition=flow_definition, title="Reconstruction flow", input_schema={})
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

## Run the Flow

Now we need to create a set of inputs for the flow, that will follow the json structure below.  The key elements that are needed are:
1. The Globus compute endpoint id
2. The Globus compute function id
3. Inputs to the Globus compute function
4. The endpoint and path from which to transfer data
5. The endpoint and path to which to transfer data

The user should choose a destination path and endpoint that they have access to.  As a sample endpoint, below the ALCF /home space is used.  To use this endpoint, the user should set a path in their home space (but remove the leading `/home` so that it will appear like `/csimpson/als_example`).

In [ ]:
alcfhome_transfer_endpoint_id = "UUID-HERE"
destination_path_on_alcfhome = "/your-path-here" # Note that paths for transfers on the home endpoint should remove the leading /home

The example problem is setup in a directory on the Eagle filesystem at ALCF, so that endpoint and path is used as the source of the data.  All the endpoints and functions are added to a `flow_input`.

In [ ]:
collection_endpoint = "UUID-HERE"
reconstruction_func = "UUID-HERE"

eagle_transfer_endpoint_id = "UUID-HERE"
source_path_on_eagle = "/IRIBeta/als/example" # Note that paths for transfers on the eagle endpoint should remove the leading /eagle

function_inputs = {"rundir": "/eagle/IRIBeta/als/bl832/raw", "file_name": "20230224_132553_sea_shell.h5", "folder_path": "BLS-00564_dyparkinson"}

flow_input = {
    "input": {
      "source": {
        "id": collection_endpoint,
        "path": "/bl832/raw/BLS-00564_dyparkinson"
      },
      "destination": {
        "id": collection_endpoint,
        "path": "/bl832tch/BLS-00564_dyparkinson"
      },
      "recursive_tx": True,
      "compute_endpoint_id": polaris_endpoint_id,
      "compute_function_id": reconstruction_func,
      "compute_function_kwargs": function_inputs
    }
}

In [ ]:
# collection_ids should contain all the transfer endpoint ids involved in the flow
collection_ids = [flow_input["input"]["source"]["id"], flow_input["input"]["destination"]["id"]]
flow_id = "UUID-HERE"
fc = get_flows_client()
flow_client = get_specific_flow_client(flow_id, collection_ids=collection_ids)

In [ ]:
flow_action = flow_client.run_flow(flow_input, label="ALS run", tags=["demo", "als", "tomopy"])
flow_run_id = flow_action['action_id']

print(f'Flow action started with id: {flow_run_id}')

print(f"Monitor your flow here: https://app.globus.org/runs/{flow_run_id}")

flow_status = flow_action['status']
# while flow_status in ['ACTIVE', 'INACTIVE']:
#     time.sleep(10)
#     flow_action = fc.get_run(flow_run_id)
#     flow_status = flow_action['status']
#     print(f'Flow status: {flow_status}')
# print(f'Final status: {flow_status}')
while flow_status in ['ACTIVE', 'INACTIVE']:
    time.sleep(10)
    flow_action = fc.get_run(flow_run_id)
    flow_status = flow_action['status']
    print(f'Updated flow status: {flow_status}')
    # Log additional details about the flow status
    print(f'Flow action details: {flow_action}')

if flow_status != 'SUCCEEDED':
    print(f'Flow failed with status: {flow_status}')
    # Log additional details about the failure
    print(f'Flow failure details: {flow_action}')
else:
    print(f'Flow completed successfully with status: {flow_status}')
    success = True


## Next Steps

Additional steps to the flow can be included if needed (for example an initial transfer step before tomopy is run).  The flow can also be adapted to be executed through the Globus web UI by adding a schema.  More information on flows can be found [here](https://docs.globus.org/guides/tutorials/flow-automation/create-a-flow/).

The Globus compute endpoint configuration can be adapted to the user's needs.  Multiple instances of tomopy can be run in parallel on each node by adapting the `config.yaml` file.

The content in this notebook can also be adapted to be run in a python script or a bash script.  More information can be found in the documentation for the [globus python api](https://globus-sdk-python.readthedocs.io/en/stable/services/flows.html) or the documentation for the [cli api](https://docs.globus.org/cli/reference/#globus_flows_commands).